# 深度學習第四次競賽報告 - 第12組 工具人智慧
組員：
<br/>107024501 高瑀鍹
<br/>107024506 王子誠
<br/>107024511 羅揚
<br/>107024522 戴子翔

# Task 1

## Data Preprocessing and Augmentation

在一個Captcha中，一張目標圖片會對應9張候選圖片，因此我們在每個epoch時，會在9張候選圖片中隨機抽其中一張以進行訓練。因為我們在task 1中並沒有使用pre-trained model，所以我們圖片的preprocessing只有將圖片normalize到0~1之間。data augmentation的部分，從圖片中可以看出正確的候選圖片為目標圖片進行旋轉過後的結果，所以我們也有對圖片進行旋轉與水平、垂直翻轉。

## Model Architecture

模型的部分是參考Hadsell et al. (2005)的論文----Dimensionality Reduction by Learning an Invariant Mapping。這個方法的主要想法是將目標圖片與候選圖片經過CNN壓縮至低維度，並在低維度之下計算兩者的相似度以比較兩張圖片是否相似，值得注意的是，兩張圖片經過的CNN的weight必須要是一致的。

架構的部分是使用第三次比賽所使用的CNN Encoder的架構，其架構為四層convolution layers，並在每個convolution layers後都使用batch normalize，再以leaky relu為activation function，最後，將先前的結果flatten後，再使用一層fully connected layer將圖片調整至128維。原先只是使用這個架構進行測試，沒想到效果意外的好 (validation準確率達99.7%)，因此最後便沒有進行額外的調整。

## Loss Function

Loss function使用的是Contrastive loss，這是由Yann LeCun與他的學生在這篇文章中所提出。
<br/> Contrastive loss的意義為，當兩個圖片為相似時，loss為兩張圖片經由network產生出的向量的距離，訓練時會讓兩者越相似；當兩個圖片不同時，loss為$max(margin-兩張圖片經由network產生出的向量的距離, 0)$，也就是將兩向量的距離反轉，訓練時會讓兩者越不相似，若兩者差距超過margin，則表示差距已經足夠大、loss為0。
<br/> 根據paper的作法，衡量兩向量差距的方式是euclidean distance，但我們使用此方法進行實作後，發現會出現loss為nan的情況，我們猜測目標圖片與候選圖片產生出的向量都沒有經過normalize，長度可能會有很大的差距導致loss出現nan。因此，我們將衡量差距的方法改為$\sqrt{(2-2cosine(target, candidate))}$，這等同於將兩向量各自normalize後再算euclidean distance，這樣調整過後就沒有發生過loss出現nan的情況。

## Other Methods

我們嘗試過其他的loss function如下：
1. Cosine loss: 若兩張圖片相似，則loss為-cosine；若不同，loss為cosine。但這個loss function的問題在於，若兩圖片不同，訓練時會讓兩者的cosine接近-1，但cosine為-1不代表向量無關，而是代表向量相反，因此，這個loss的設計並不合理，效果也沒有contrastive loss好，validation準確率約85%。
2. Triplet loss: 這種方法將三張圖片（anchor, positive, negative. 其中positive和anchor是相同的物品；negative和anchor是不同的物品）, 分別送進三個相同的神經網路。訓練目標是希望讓positive和anchor在output的距離越近越好，而negative和anchor在output的距離越遠越好。

除了訓練模型的方法之外，我們也突發奇想的藉由vgg19，先將目標圖片輸出所對應最有可能的3種類別，再同樣地對應9張照片也可以得出最有可能的3種類別，若目標圖片的前3種類別有出現在所對應的9張照片的前三種類別中，我們就會預測這兩張照片應該是同樣的，但效果並不好，validation準確率約60%。